In [30]:
%reload_ext autoreload
%autoreload 2

In [31]:
from pathlib import Path

# custom module
import sys

sys.path.append("../../../")  # include parent directory
from src.ookla import *

In [32]:
ADMIN_FPATH = Path("../../../data/01-admin-bounds/")
OUTPUT_DIR = Path("../../../data/04-output/ookla/")
RAW_DIR = Path("../../../data/02-raw/")
OOKLA_DIR = Path("../../../data/02-raw/ookla/")

OOKLA_YEARS = ["2022", "2021", "2020", "2019"]

### Load AOI

In [33]:
# Load the administrative boundaries for the 12 cities
aoi = gpd.read_file(ADMIN_FPATH / "target_admin_bounds.shp")
aoi.head(2)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."


# Align Ookla data to AOI

`OOKLA_DIR` contains the downloaded raw ookla files from the previous notebook.

In [34]:
def align_ookla(year="2022"):
    ookla_data_manager = OoklaDataManager(cache_dir=RAW_DIR)
    ookla_features = add_ookla_features(aoi, "fixed", year, ookla_data_manager)
    ookla_features_mobile = add_ookla_features(aoi, "mobile", year, ookla_data_manager)
    drop_cols = [
        "ADM1_EN",
        "ADM1_PCODE",
        "ADM2_EN",
        "ADM2_PCODE",
        "ADM3_EN",
        "ADM3_PCODE",
        "ADM4_EN",
        "geometry",
    ]
    ookla_features = ookla_features.drop(columns=drop_cols)
    ookla_features_mobile = ookla_features_mobile.drop(columns=drop_cols)

    ookla_df = ookla_features.merge(ookla_features_mobile, on="ADM4_PCODE")
    ookla_df.insert(1, "date", f"{year}-10-01")
    ookla_df.insert(1, "freq", "Y")

    ookla_df.to_csv(OUTPUT_DIR / f"ookla_features_{year}.csv")
    return ookla_df

In [35]:
for year in OOKLA_YEARS:
    align_ookla(year)

2024-02-01 11:57:20.406 | DEBUG    | src.ookla:load_type_year_data:76 - Contents of data cache: []
2024-02-01 11:57:20.409 | INFO     | src.ookla:load_type_year_data:91 - Cached data available at ../../../data/02-raw/ookla/processed/afc08259ba9585cdf5ee4494d1e49fbd.csv? True
2024-02-01 11:57:20.410 | DEBUG    | src.ookla:load_type_year_data:96 - Processed Ookla data for aoi, fixed 2022 (key: afc08259ba9585cdf5ee4494d1e49fbd) found in filesystem. Loading in cache.


2024-02-01 11:57:21.277 | DEBUG    | src.ookla:load_type_year_data:76 - Contents of data cache: ['afc08259ba9585cdf5ee4494d1e49fbd']
2024-02-01 11:57:21.278 | INFO     | src.ookla:load_type_year_data:91 - Cached data available at ../../../data/02-raw/ookla/processed/588422765f080d117cc64adbde2a3582.csv? True
2024-02-01 11:57:21.279 | DEBUG    | src.ookla:load_type_year_data:96 - Processed Ookla data for aoi, mobile 2022 (key: 588422765f080d117cc64adbde2a3582) found in filesystem. Loading in cache.
2024-02-01 11:57:22.216 | DEBUG    | src.ookla:load_type_year_data:76 - Contents of data cache: []
2024-02-01 11:57:22.218 | INFO     | src.ookla:load_type_year_data:91 - Cached data available at ../../../data/02-raw/ookla/processed/b79316ea94d6c1e846ac36e032567763.csv? True
2024-02-01 11:57:22.218 | DEBUG    | src.ookla:load_type_year_data:96 - Processed Ookla data for aoi, fixed 2021 (key: b79316ea94d6c1e846ac36e032567763) found in filesystem. Loading in cache.
2024-02-01 11:57:23.179 | DEB

2024-02-01 11:58:58.411 | INFO     | src.ookla:download_ookla_year_data:249 - Ookla Data: Downloading Ookla parquet file for quarter 2...
2024-02-01 11:58:59.524 | INFO     | geowrangler.datasets.utils:urlretrieve:26 - Retrieving https://ookla-open-data.s3.us-west-2.amazonaws.com/parquet/performance/type=mobile/year=2021/quarter=2/2021-04-01_performance_mobile_tiles.parquet into ../../../data/02-raw/ookla/mobile/2021/2021-04-01_performance_mobile_tiles.parquet


2024-02-01 12:00:34.417 | INFO     | src.ookla:download_ookla_year_data:249 - Ookla Data: Downloading Ookla parquet file for quarter 3...
2024-02-01 12:00:35.497 | INFO     | geowrangler.datasets.utils:urlretrieve:26 - Retrieving https://ookla-open-data.s3.us-west-2.amazonaws.com/parquet/performance/type=mobile/year=2021/quarter=3/2021-07-01_performance_mobile_tiles.parquet into ../../../data/02-raw/ookla/mobile/2021/2021-07-01_performance_mobile_tiles.parquet


2024-02-01 12:02:03.825 | INFO     | src.ookla:download_ookla_year_data:249 - Ookla Data: Downloading Ookla parquet file for quarter 4...
2024-02-01 12:02:04.857 | INFO     | geowrangler.datasets.utils:urlretrieve:26 - Retrieving https://ookla-open-data.s3.us-west-2.amazonaws.com/parquet/performance/type=mobile/year=2021/quarter=4/2021-10-01_performance_mobile_tiles.parquet into ../../../data/02-raw/ookla/mobile/2021/2021-10-01_performance_mobile_tiles.parquet


...

2024-02-01 12:09:02.523 | INFO     | src.ookla:download_ookla_year_data:259 - Ookla Data: Successfully downloaded and cached Ookla data for mobile and 2020 at ../../../data/02-raw/ookla/mobile/2020!
2024-02-01 12:09:02.524 | DEBUG    | src.ookla:load_type_year_data:143 - Generating quadkeys based on input aoi geometry to pull intersecting Ookla data.
2024-02-01 12:09:32.861 | DEBUG    | src.ookla:load_type_year_data:160 - Ookla data for aoi, mobile 2020 1 being loaded from ../../../data/02-raw/ookla/mobile/2020/2020-01-01_performance_mobile_tiles.parquet
2024-02-01 12:09:35.883 | DEBUG    | src.ookla:load_type_year_data:160 - Ookla data for aoi, mobile 2020 2 being loaded from ../../../data/02-raw/ookla/mobile/2020/2020-04-01_performance_mobile_tiles.parquet
2024-02-01 12:09:38.825 | DEBUG    | src.ookla:load_type_year_data:160 - Ookla data for aoi, mobile 2020 3 being loaded from ../../../data/02-raw/ookla/mobile/2020/2020-07-01_performance_mobile_tiles.parquet
2024-02-01 12:09:42.717

2024-02-01 12:11:03.980 | INFO     | src.ookla:download_ookla_year_data:249 - Ookla Data: Downloading Ookla parquet file for quarter 2...
2024-02-01 12:11:05.122 | INFO     | geowrangler.datasets.utils:urlretrieve:26 - Retrieving https://ookla-open-data.s3.us-west-2.amazonaws.com/parquet/performance/type=mobile/year=2019/quarter=2/2019-04-01_performance_mobile_tiles.parquet into ../../../data/02-raw/ookla/mobile/2019/2019-04-01_performance_mobile_tiles.parquet


2024-02-01 12:12:09.495 | INFO     | src.ookla:download_ookla_year_data:249 - Ookla Data: Downloading Ookla parquet file for quarter 3...
2024-02-01 12:12:10.544 | INFO     | geowrangler.datasets.utils:urlretrieve:26 - Retrieving https://ookla-open-data.s3.us-west-2.amazonaws.com/parquet/performance/type=mobile/year=2019/quarter=3/2019-07-01_performance_mobile_tiles.parquet into ../../../data/02-raw/ookla/mobile/2019/2019-07-01_performance_mobile_tiles.parquet


2024-02-01 12:13:22.262 | INFO     | src.ookla:download_ookla_year_data:249 - Ookla Data: Downloading Ookla parquet file for quarter 4...
2024-02-01 12:13:23.441 | INFO     | geowrangler.datasets.utils:urlretrieve:26 - Retrieving https://ookla-open-data.s3.us-west-2.amazonaws.com/parquet/performance/type=mobile/year=2019/quarter=4/2019-10-01_performance_mobile_tiles.parquet into ../../../data/02-raw/ookla/mobile/2019/2019-10-01_performance_mobile_tiles.parquet


2024-02-01 12:14:55.395 | INFO     | src.ookla:download_ookla_year_data:259 - Ookla Data: Successfully downloaded and cached Ookla data for mobile and 2019 at ../../../data/02-raw/ookla/mobile/2019!
2024-02-01 12:14:55.398 | DEBUG    | src.ookla:load_type_year_data:143 - Generating quadkeys based on input aoi geometry to pull intersecting Ookla data.
2024-02-01 12:15:25.908 | DEBUG    | src.ookla:load_type_year_data:160 - Ookla data for aoi, mobile 2019 1 being loaded from ../../../data/02-raw/ookla/mobile/2019/2019-01-01_performance_mobile_tiles.parquet
2024-02-01 12:15:28.355 | DEBUG    | src.ookla:load_type_year_data:160 - Ookla data for aoi, mobile 2019 2 being loaded from ../../../data/02-raw/ookla/mobile/2019/2019-04-01_performance_mobile_tiles.parquet
2024-02-01 12:15:30.732 | DEBUG    | src.ookla:load_type_year_data:160 - Ookla data for aoi, mobile 2019 3 being loaded from ../../../data/02-raw/ookla/mobile/2019/2019-07-01_performance_mobile_tiles.parquet
2024-02-01 12:15:33.729